# Prediction Models

Ensemble-Methoden:  
•	AdaBoostClassifier  
•	AdaBoostRegressor  
•	BaggingClassifier  
•	BaggingRegressor  
•	GradientBoostingClassifier  
•	GradientBoostingRegressor  
•	RandomForestClassifier  
•	RandomForestRegressor  
•	VotingClassifier  
•	VotingRegressor  
•	StackingClassifier   
•	StackingRegressor   

Lineare Modelle:  
•   LinearRegression  
•	LogisticRegression  
•	LogisticRegressionCV  
•	Ridge  
•	RidgeClassifier  
•	RidgeClassifierCV  
•	RidgeCV  
•	Lasso  
•	LassoCV  
•	ElasticNet  
•	ElasticNetCV  
•	OrthogonalMatchingPursuit  
•	OrthogonalMatchingPursuitCV  

Naive Bayes:  
•	BernoulliNB  
•	ComplementNB  
•	GaussianNB  
•	MultinomialNB  

Nearest Neighbors:  
•	KNeighborsClassifier  
•	KNeighborsRegressor  
•	RadiusNeighborsClassifier  
•	RadiusNeighborsRegressor  

Decision Trees:  
•	DecisionTreeClassifier  
•	DecisionTreeRegressor  
•	ExtraTreeClassifier  
•	ExtraTreeRegressor  

Support Vector Machines:  
•	LinearSVC  
•	LinearSVR  
•	NuSVC  
•	NuSVR  
•	OneClassSVM  
•	SVC  
•	SVR  

Neural Networks:  
•	MLPClassifier  
•	MLPRegressor  

Clustering:  
•	KMeans  
•	AgglomerativeClustering  
•	Birch  
•	DBSCAN  
•	FeatureAgglomeration  
•	MeanShift  
•	MiniBatchKMeans  
•	OPTICS  
•	SpectralClustering  

Gaussian Process:  
•	GaussianProcessClassifier  
•	GaussianProcessRegressor  

Semi-Supervised Learning:  
•	LabelPropagation  
•	LabelSpreading  